# NHK News Easy Hard Classifier

## Description
This model aims to classify a given input text in Japanese into one of two reading comprehension difficulty levels: easy or hard.

TODO:
- describe the dataset used
- describe the transformation pipeline
- describe the training methodology
- describe the deployment strategy
- conclude with interesting observations

In [1]:
# Step 1. Load the dataset and annotate the training data
import pandas as pd

easy_news = pd.read_csv("../resources/nhk_news/easy_articles.csv")
hard_news = pd.read_csv("../resources/nhk_news/hard_articles.csv")

easy_texts = easy_news['texts']
hard_texts = hard_news['texts']

EASY = 1
HARD = -1
easy_targets = pd.Series(len(easy_texts) * [EASY])
hard_targets = pd.Series(len(hard_texts) * [HARD])

X = pd.concat([easy_texts,   hard_texts],   ignore_index=True)
y = pd.concat([easy_targets, hard_targets], ignore_index=True)

In [5]:
# Step 2. Replace each word with its embedding
# TODO:
# - Tokenize
# - Get embedding for each word
# - Pad each text

In [ ]:
# Step 3. Split the dataset into a training and validation set
from sklearn.model_selection import train_test_split

train_X, train_y, val_X, val_y = train_test_split(X, y, test_size=0.2, random_state=1)